<div style="text-align:center"><span style="font-size:1.5cm; line-height: 2cm; font-family:cursive; ">顧客推薦清單<br>與模型檔案製作</span><br></div>

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">一 、 載入套件與資料</span><br>

In [ ]:
# Importing the libraries
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from util import get_dummies, detect_str_columns,model_testRF,results_summary_to_dataframe,plot_confusion_matrix,logistic_model,logistic_importance,logistic_conf

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, auc, accuracy_score,classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# ----設定繪圖-------
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import os
import platform

# 讀取電商資料
data = pd.read_csv('contract.csv')

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">二 、 變數視覺化</span><br>

In [ ]:
# Buy 比例
print('不購買', round(data['buy'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('購買', round(data['buy'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

# 看看 y變數的分佈圖
colors = ["#0101DF", "#DF0101"]
sns.countplot('buy', data=data, palette=colors)
plt.title('purchase decision \n (0:no vs 1:yes )', fontsize=14)

# 不平衡資料集 --> imbalanced dataset

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">三 、 資料處理與轉換</span><span style="font-size:0.8cm; line-height: 2cm; font-family:cursive;"> - 變數轉換</span><br>

In [ ]:
'''
 1. 標籤編碼（Label Encoding）：該方法將各類別轉換成數字，在商業界及社會科學中，
                                最常見的就是李克特5點量表的轉換，這也是「順序尺
                                度」之概念，也就是具有大小之分的類別，建議適用該
                                法。
                                
 2. 獨熱編碼（One-Hot Encoding）：這代表將類別轉換成欄位格式，並且以1（有該類別）
                                與0（沒有該類別）的形式表示之。
''' 

<span style="font-size:0.7cm;font-family:cursive;"> 1. 使用 - 獨熱編碼</span><br>

In [ ]:
# 偵測有字串的欄位
str_columns = detect_str_columns(data)

print(str_columns)
'''
why? 
因為['使用者地區', '性別']皆非順序尺度，當作一般one hot encoding類別變數處理就可
'''
dataset = get_dummies(['locations'], data)
dataset.head(5)

<span style="font-size:0.7cm;font-family:cursive;"> 3. 資料處理與轉換 - 小練習</span><br>

<span style="font-size:0.6cm;font-family:cursive;">問題一</span><br>
<span style="font-size:0.6cm;font-family:cursive;">請問我們應該如何將「性別 gender」也變成one hot encoding的變數？</span><br> 

<span style="font-size:0.6cm;font-family:cursive;">問題二</span><br>
<span style="font-size:0.6cm;font-family:cursive;">請問我們應該如何「自動套入」get_dummies，讓`['locations', 'gender']`同時都變成one hot encoding 的形式?</span><br> 

<span style="font-size:0.6cm;font-family:cursive;">作答 : </span><br>

In [ ]:

# 轉換完one-hot encoding後，要去查看info
# why? 因為電腦只能判讀數字，要查看Dtype是否轉換成
# 整數（int） or 浮點數（float）
# 這樣電腦就可以計算囉！
dataset.info()

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">四 、 資料處理與轉換</span><span style="font-size:0.8cm; line-height: 2cm; font-family:cursive;"> - 切分資料集</span><br>

<span style="font-size:0.7cm;font-family:cursive;"> 1. 將 x 與 y 分割開來</span><br>

In [ ]:
# 將X與y分割開來
X =dataset.drop(columns=['buy'])
y =dataset['buy']

<span style="font-size:0.7cm;font-family:cursive;"> 2. 切分資料集</span><br>

In [ ]:
# 切分資料集

'''
切分成80％的訓練資料集
    - X_train : X訓練變數
    - y_train : y訓練變數
    
切分成20％的測試資料集
    - X_test : X測試變數
    - y_test : y測試變數
'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 


<span style="font-size:0.7cm;font-family:cursive;"> 3. 查看各自的維度</span><br>

In [ ]:
# X 訓練資料集維度
X_train.shape

In [ ]:
# y 訓練資料集維度
y_train.shape

In [ ]:
# X 測試資料集維度
X_test.shape

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">四 、 資料處理與轉換</span><span style="font-size:0.7cm; line-height: 2cm; font-family:cursive;"> - 將UID拿出來</span><br>

<span style="font-size:0.7cm;font-family:cursive;"> 1. 保留UID</span><br>

In [ ]:
# 保留UID  做推薦清單時需要將uid放回去 找到推薦人
train_uid = X_train['UID']
test_uid = X_test['UID']

<span style="font-size:0.7cm;font-family:cursive;"> 2. 刪除UID</span><br>

In [ ]:
# 刪除UID 流水號
del X_train['UID']
#X_train['UID']
del X_test['UID']

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">五 、 羅吉斯迴歸</span>

<span style="font-size:0.7cm;font-family:cursive;"> 1. 命名模型物件</span><br>

In [ ]:
logistic_reg =LogisticRegression()

<span style="font-size:0.7cm;font-family:cursive;"> 2. Feature Scaling</span><br>

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_logistic = sc.fit_transform(X_train)
X_test_logistic = sc.transform(X_test)

<span style="font-size:0.7cm;font-family:cursive;"> 3. 進行訓練</span><br>

In [ ]:
logistic_reg.fit(X_train_logistic, y_train)

<span style="font-size:0.7cm;font-family:cursive;"> 4. 預測</span><br>

In [ ]:
logistic_pred = logistic_reg.predict(X_test_logistic)

<span style="font-size:0.7cm;font-family:cursive;"> 5. 預測機率</span><br>

In [ ]:
logistic_pred_proba = logistic_reg.predict_proba(X_test_logistic) 

<span style="font-size:0.7cm;font-family:cursive;"> 6. 準確率</span><br>

In [ ]:
accuracy_score(y_test, logistic_pred)

<span style="font-size:0.7cm;font-family:cursive;"> 7. confusion matrix plot</span><br>

In [ ]:
model_testRF( LogisticRegression(), 
             X_train_logistic,y_train,X_test_logistic,y_test, plot_name = 'LogisticRegression')

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">六 、 隨機森林</span>


In [ ]:
'''
將模型物件命名：rf_model
給出100棵樹進行預測: RandomForestClassifier(n_estimators = 100, random_state = 0,verbose=1,n_jobs=8)

'''

<span style="font-size:0.7cm;font-family:cursive;"> 1. 命名模型物件</span><br>

In [ ]:
rf_model =RandomForestClassifier(n_estimators = 100, random_state = 0,verbose=1,n_jobs=8)

<span style="font-size:0.7cm;font-family:cursive;"> 2. Feature Scaling</span><br>

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_logistic = sc.fit_transform(X_train)
X_test_logistic = sc.transform(X_test)

<span style="font-size:0.7cm;font-family:cursive;"> 3. 進行訓練</span><br>

In [ ]:
rf_model.fit(X_train_logistic, y_train)

<span style="font-size:0.7cm;font-family:cursive;"> 4. 預測</span><br>

In [ ]:
rf_pred = rf_model.predict(X_test_logistic)

<span style="font-size:0.7cm;font-family:cursive;"> 5. 預測機率</span><br>

In [ ]:
rf_pred_proba = rf_model.predict_proba(X_test_logistic) 

<span style="font-size:0.7cm;font-family:cursive;"> 6. 準確率</span><br>

In [ ]:
accuracy_score(y_test, rf_pred)

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">七 、 XGBoost練習</span>


In [ ]:
'''
xgb_model
1. 給出100棵樹進行預測: xgb_model = XGBClassifier(n_estimators=100 ,random_state = 0,
 nthread = 8, learning_rate=0.5)

2. fit部分請改：fit(X_train, y_train, verbose=True,
                          eval_set=[(X_train, y_train), (X_test, y_test)])

3. rf的名稱部分全部改成xgb

'''

<span style="font-size:0.7cm;font-family:cursive;"> 問題1 : 命名模型物件</span><br>

<span style="font-size:0.7cm;font-family:cursive;"> 問題2 : 進行訓練</span><br>

<span style="font-size:0.7cm;font-family:cursive;"> 作答:</span><br>

<span style="font-size:0.7cm;font-family:cursive;"> 3. 預測</span><br>

In [ ]:
xgb_pred = xgb_model.predict(X_test)

<span style="font-size:0.7cm;font-family:cursive;"> 4. 預測機率</span><br>

In [ ]:
xgb_pred_proba = xgb_model.predict_proba(X_test) 

<span style="font-size:0.7cm;font-family:cursive;"> 5. 準確率</span><br>

In [ ]:
accuracy_score(y_test, xgb_pred)

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">七 、 製作顧客產品推薦名單</span>


In [ ]:
XGBClassifier_test_df=pd.DataFrame(y_test.values ,columns =['客戶對A商品【實際】購買狀態'])
XGBClassifier_test_df['客戶對A商品【預測】購買機率'] = xgb_pred_proba[:,1]
test_uid = test_uid.reset_index().drop(columns = ['index'])  

<span style="font-size:0.7cm;font-family:cursive;"> 問題1 : <br> 如何將test_uid與XGBClassifier_test_df結合成2欄</span><br>

In [ ]:
# hint : concat
XGBClassifier_test_df=pd.concat([？？？], 
          axis = ？？？)

<span style="font-size:0.7cm;font-family:cursive;"> 問題2 : <br>根據'客戶對A商品【預測】購買機率'排序</span><br>

In [ ]:
# hint : sort_values()
XGBClassifier_test_df = XGBClassifier_test_df.？？？

<span style="font-size:0.7cm;font-family:cursive;"> 問題3 : <br> 輸出csv，命名為顧客產品推薦名單.csv</span><br>

In [ ]:
XGBClassifier_test_df.？？？

<span style="font-size:0.7cm;font-family:cursive;"> 如果是驗證資料 model, y_test, X_test, test_uid</span><br>

In [ ]:
from util import client_list
client_list(model = xgb_model,y_test=y_test, X_test=X_test, test_uid=test_uid, name = 'test')

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">八 、<br> 針對未知/新資料的預測資料實戰</span>


In [ ]:
'''
前處理
1. 讀取資料 (pd.read_csv)
2. 保留UID ( newdata['UID'])
3. 從原始資料中刪除UID (del newdata['UID'])
4. 產出【顧客產品推薦名單】 (client_list() )
'''

<span style="font-size:0.7cm;font-family:cursive;"> 1. 讀取資料</span><br>

In [ ]:
newdata = pd.read_csv('contract_newdata.csv')

<span style="font-size:0.7cm;font-family:cursive;"> 2. 保留UID</span><br>

In [ ]:
newdata_uid = newdata['UID']

<span style="font-size:0.7cm;font-family:cursive;"> 3. 刪除UID</span><br>

In [ ]:
del newdata['UID']

<span style="font-size:0.7cm;font-family:cursive;"> 4. 產出【顧客產品推薦名單】</span><br>

In [ ]:
c_list = client_list(model=xgb_model, y_test=None, X_test=newdata, test_uid=newdata_uid, name ='new')

<span style="font-size:0.7cm;font-family:cursive;"> 問題 : 針對未知的預測資料實戰練習</span><br>

In [ ]:
'''
目前有新進的資料
U1.csv
U2.csv

如何判斷他們的購買機率？ 並產出個別檔案的【顧客產品推薦名單】
'''

<span style="font-size:0.7cm;font-family:cursive;"> 作答:</span><br>

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">九 、儲存model</span>


<span style="font-size:0.7cm;font-family:cursive;"> 1. 載入 pickle</span><br>

In [ ]:
import pickle

<span style="font-size:0.7cm;font-family:cursive;"> 2. 儲存模型 pickle</span><br>

In [ ]:
pickle.dump(xgb_model, open("xgb_model.dat", "wb"))  #wb : write binary (寫入成二元檔案)

xgb_model
del xgb_model
# xgb_model

<span style="font-size:0.7cm;font-family:cursive;"> 3. 載入模型，做精準名單 pickle</span><br>

In [ ]:
model_xgb = pickle.load(open("xgb_model.dat", "rb"))

c_list2 = client_list(model=model_xgb, y_test=None, X_test=newdata, test_uid=newdata_uid,name ='model')

<span style="font-size:1cm; line-height: 2cm; font-family:cursive;">十 、<br>製作顧客產品推薦名單 function</span>

In [ ]:
'''
input: data

output: 顧客產品推薦名單
'''
from util import client_list_all

<span style="font-size:0.5cm;font-family:cursive;"> 1. 顧客產品推薦名單 function</span><br>
<span style="font-size:0.5cm;font-family:cursive;"> 2. model_name: finder裡面有的model存儲檔案load進來</span><br>
<span style="font-size:0.5cm;font-family:cursive;"> 3. dataname: 新的資料</span><br>
<span style="font-size:0.5cm;font-family:cursive;"> 4. name: 存出來的檔案名稱
</span><br>

In [ ]:
final_list = client_list_all(model_name="xgb_model.dat",
                             dataname='contract_newdata.csv',
                             name='new2' )
final_list